# FUNCTION glcm

<strong> 2D offsets </strong>

|Directions| Degrees |
|-----------|-------------|
| [0, D]   | zero degrees |
| [D, -D]  | 45 degrees   |          
| [D, 0]   | 90 degrees   | 
| [D, D]   | 135 degrees  | 

<strong> 3D offsets </strong>

Standard 2D directions

|Directions| Degrees |
|-----------|-------------|
| [0, D]   | zero degrees |
| [D, -D]  | 45 degrees   |          
| [D, 0]   | 90 degrees   | 
| [D, D]   | 135 degrees  | 

Additional 9 directions


| Directions  | Degrees zy  | Degrees yx  |
|-------------|-------------|-------------|
| [D 0 -D]    | 0 degrees   | 45 degrees  |
| [D 0 0]     | straight up |             |
| [D 0 D]     | 0 degrees   | 135 degrees |
| [D D 0]     | 90 degrees  | 45 degrees  |
| [D -D 0]    | 90 degrees  | 135 degrees |
| [D D -D]    | 45 degrees  | 45 degrees  |
| [D -D D]    | 45 degrees  | 135 degrees |
| [D D D]     | 135 degrees | 45 degrees  |
| [D -D -D]   | 135 degrees | 135 degrees |


In [53]:
import numpy as np

# 2D implementations

def glcm2d_c(levels, arr=[], offset=[], result=[]): # function previously implemented in C
    x, y = arr.shape
    if np.all(offset ==0):
        h = offset.shape
    h1, w1 = result.shape
    
    for i in range (0, x):
        for j in range(0, y):
            aux = arr[i][j]
            rows = i+offset[0]
            cols = j+offset[1]
            if(rows >= 0 and rows < x):
                if(cols < y and cols >= 0):                    
                    aux2 = arr[rows][cols]                    
                    if(aux >= 0 and aux < levels):
                        if(aux2 >= 0 and aux2 < levels):
                            result[aux][aux2]= result[aux][aux2]+1
    return result 

def glcm2(levels, offsets, img):
    result = np.zeros((levels, levels), dtype=np.uint32)
    result = glcm2d_c(levels, img, offset, result)
    return result

# 3D implementations

def glcm3d_c(levels, arr=[], offset=[], result=[]):
    z, x, y= arr.shape
    h = offset.shape
    h1, w1 = result.shape
    for k in range(0, z):
        for i in range(0, x):
            for j in range(0, y):
                aux = arr[k][i][j]
                slices = k+offset[0]
                rows = i+offset[1]
                cols = j+offset[2]
                if(slices >= 0 and slices < z):
                    if(rows >=0 and rows < x):
                        if(cols >=0 and cols<y):
                            aux2 = arr[slices][rows][cols]
                            if(aux >=0 and aux<levels):
                                if(aux2 >=0 and aux2 < levels):
                                    result[aux][aux2] = result[aux][aux2]+1
    return result

def glcm3(levels, offset, img):
    result = np.zeros((levels,levels), dtype = np.int32)
    result = glcm3d_c(levels, img, offset, result)
    return result

def glcm(levels, offsets, img):
    print 'entrei glcm'
    result = np.zeros((levels, levels), dtype = np.int32)
    print 'result created: ok!'
    if len(img.shape)==2:
        print 'image shape: 2'
        if np.all(offsets==0):
            print 'offsets in 2d case DOES NOT EXIST'
            offsets = np.array([0,1], dtype = np.int32)
        glcm2d_c(levels, img, offsets, result)
    elif len(img.shape) == 3:
        print 'image shape 3'
        if np.all(offsets==0):
            print 'offsets in 3d case does not EXIST'
            offsets = np.array([0,0,1], dtype = np.int32)
        glcm3d_c(levels, img, offsets, result)
    else:
        print 'Invalid Input, Please use another array'
    return result

# EXAMPLES

<strong> 2D Examples </strong>

In [54]:
arr = np.array([[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]], dtype = np.uint8)
levels = int(arr.max()+1)
offset = np.array([0,1], dtype = np.int32) # orientation 0 degrees
result = glcm2(levels, offset, arr)
print result
print
offset = np.array([1,-1], dtype = np.int32) # orientation 45 degrees
result = glcm2(levels, offset, arr)
print result
print
offset = np.array([1,0], dtype = np.int32) # orientation 90 degrees
result = glcm2(levels, offset, arr)
print result
print
offset = np.array([1,1], dtype = np.int32) # orientation 135 degrees
result = glcm2(levels, offset, arr)
print result
print

[[5 2 1]
 [1 4 4]
 [2 1 5]]

[[0 4 2]
 [1 4 1]
 [5 0 3]]

[[0 3 4]
 [2 5 2]
 [5 1 2]]

[[1 2 4]
 [2 3 2]
 [3 1 2]]



<strong> 3D Examples </strong>

Standard directions

In [55]:
arr = np.zeros((3,5,6), dtype=np.uint8)
arr = np.array([[[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]],
                [[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]],
                [[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]]], dtype=np.uint8)
levels = int(arr.max()+1)

offset = np.array([0,0,1], dtype = np.int32) # orientation 0 degrees (example same slices: equal to Nslices*0degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([0,1,-1], dtype = np.int32) # orientation 45 degrees (example same slices: equal to Nslices*45degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([0,1,0], dtype = np.int32) # orientation 90 degrees (example same slices: equal to Nslices*90degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([0,1,1], dtype = np.int32) # orientation 135 degrees (example same slices: equal to Nslices*135degree 2D case)
result = glcm3(levels, offset, arr)
print result
print

[[15  6  3]
 [ 3 12 12]
 [ 6  3 15]]

[[ 0 12  6]
 [ 3 12  3]
 [15  0  9]]

[[ 0  9 12]
 [ 6 15  6]
 [15  3  6]]

[[ 3  6 12]
 [ 6  9  6]
 [ 9  3  6]]



Additional 9 directions

In [56]:
offset = np.array([1,0,-1], dtype = np.int32) # 0 degrees/45 degrees (example same slices: equal to (Nslices-1)*0degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([1,0,0], dtype = np.int32) # straight up (example same slices: equal to np.unique())
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([1,0,1], dtype = np.int32) # 0 degree/135 degrees (example same slices: equal to (Nslices-1)*transpose of 0degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([1,1,0], dtype = np.int32) # 90 degrees/45 degrees (example same slices: equal to (Nslices-1)*90 degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([1,-1,0], dtype = np.int32) # 90 degrees/135 degrees (example same slices: equal to (Nslices-1)*transpose of 90 degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([1,1,-1], dtype = np.int32) # 45 degrees/45 degrees (example same slices: equal to (Nslices-1)*45 degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([1,-1,1], dtype = np.int32) # 45 degree/135 degrees (example same slices: equal to (Nslices-1)*transpose of 45 degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([1,1,1], dtype = np.int32) # 135 degrees/45 degrees (example same slices: equal to (Nslices-1)*135 degree 2D case)
result = glcm3(levels, offset, arr)
print result
print
offset = np.array([1,-1,-1], dtype = np.int32) # 135 degrees/135 degrees (example same slices: equal to (Nslices-1)*transpose of 135 degree 2D case)
result = glcm3(levels, offset, arr)
print result
print

[[10  2  4]
 [ 4  8  2]
 [ 2  8 10]]

[[18  0  0]
 [ 0 22  0]
 [ 0  0 20]]

[[10  4  2]
 [ 2  8  8]
 [ 4  2 10]]

[[ 0  6  8]
 [ 4 10  4]
 [10  2  4]]

[[ 0  4 10]
 [ 6 10  2]
 [ 8  4  4]]

[[ 0  8  4]
 [ 2  8  2]
 [10  0  6]]

[[ 0  2 10]
 [ 8  8  0]
 [ 4  2  6]]

[[2 4 8]
 [4 6 4]
 [6 2 4]]

[[2 4 6]
 [4 6 2]
 [8 4 4]]



In [57]:
import nbimporter
from glcmdesc import *
f = np.array( [[2,1,1,0,0],
               [1,2,0,0,0],
               [1,0,2,1,0],
               [0,0,0,2,0]], dtype=np.uint8)
offset = np.array([1,1],dtype=np.int32)
glcm, v = glcmdesc(f,offset)
print glcm
print 'glcm descriptors',v

entrei glcm
result created: ok!
image shape: 2
[[3 1 0]
 [5 0 0]
 [0 0 3]]
entrei glcm
result created: ok!
image shape: 2
[[ 0.25        0.08333333  0.        ]
 [ 0.41666667  0.          0.        ]
 [ 0.          0.          0.25      ]]
glcm descriptors [ 0.30555556  0.5         0.71095787  0.57638889  0.75        3.5         2.25
  0.45154499  0.54938312  0.25        0.30103   ]


In [58]:
f  = np.array( [[1,2,2,2],
             [2,2,1,1],
             [1,2,0,1],
             [2,1,0,1]], dtype=np.uint8)
mask = f!=1
print mask
glcm,v = glcmdesc(f, mask)
print 'glcm: ', glcm
print 'glcm descriptors',v

[[False  True  True  True]
 [ True  True False False]
 [False  True  True False]
 [ True False  True False]]
entrei glcm
result created: ok!
image shape: 2


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()